In [28]:
# print("Installing Pipeline...")
# !pip install pipeline
# print("Installing Transformers...")
# !pip install transformers
# print("Installing TensorFlow...")
# !pip install tensorflow
# print("Installing Keras...")
# !pip install tf-keras
# print("All packages installed...")
from transformers import pipeline
import pandas as pd

In [31]:
distilled_student_sentiment_classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment",
    #Best accuracy ="cardiffnlp/twitter-roberta-base-sentiment-latest",
    #Second Best = "lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    #Replace inside the model = "" to change what type of model is used.
    # top_k= None,
    top_k=1,
    model_kwargs={'max_length': 2512}
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
print(distilled_student_sentiment_classifier.model.config.max_length)

2512


In [33]:
print(distilled_student_sentiment_classifier("i love you"))

[[{'label': 'positive', 'score': 0.8294013142585754}]]


In [34]:
df = pd.read_csv("./Datasets/fake_or_real_news.csv")

In [35]:
# #Usage for unlimited text size
# i = 0
# for texts in df['text']:
#     label = distilled_student_sentiment_classifier(texts)
# 
#     print("ID: ",i," Label: ",label)
#     i+= 1

In [36]:
def predict_long_text(text, window_size=512, overlap=100):
    posC = 0
    negC = 0
    neutralC = 0
    max_score = 0  # Initialize max_score
    max_label = ''

    chunks = [text[i:i+window_size] for i in range(0, len(text), window_size - overlap)]
    flattened_sentiments = [sentiment for chunk in chunks for sentiment in distilled_student_sentiment_classifier(chunk)]

    for result in flattened_sentiments:
        for sentiment in result:
            label = sentiment['label']
            score = sentiment['score']
            if label == 'positive':
                posC += 1
            elif label == 'negative':
                negC += 1
            elif label == 'neutral':
                neutralC += 1
            if score > max_score:  # Check if the score is higher
                max_score = score
                max_label = label

    # Determine which count is the highest
    max_count = max(posC, negC, neutralC)

    # Get the label corresponding to the highest count
    if max_count == posC:
        max_label = 'positive'
    elif max_count == negC:
        max_label = 'negative'
    else:
        max_label = 'neutral'

    return max_label, max_score  # Return both the label and the score
# Example usage:
sentiment_labels = []
i = 0
for texts in df['text']:
    sentiment, score = predict_long_text(texts)
    label = df['label'][i]
    # if score <= 0.8:
    #     sentiment = "neutral"
    print("Reality: ",label,"ID: ",i," Label: ",sentiment, " Accuracy: ",score)
    i+= 1

Reality:  FAKE ID:  0  Label:  negative  Accuracy:  0.9100181460380554
Reality:  FAKE ID:  1  Label:  negative  Accuracy:  0.9156620502471924
Reality:  REAL ID:  2  Label:  neutral  Accuracy:  0.9321503043174744
Reality:  FAKE ID:  3  Label:  neutral  Accuracy:  0.7992492318153381
Reality:  REAL ID:  4  Label:  neutral  Accuracy:  0.9191883206367493
Reality:  FAKE ID:  5  Label:  negative  Accuracy:  0.929938018321991
Reality:  FAKE ID:  6  Label:  neutral  Accuracy:  0.9200218319892883
Reality:  REAL ID:  7  Label:  neutral  Accuracy:  0.885099470615387
Reality:  REAL ID:  8  Label:  neutral  Accuracy:  0.9139233231544495
Reality:  REAL ID:  9  Label:  neutral  Accuracy:  0.9192206263542175
Reality:  REAL ID:  10  Label:  neutral  Accuracy:  0.9080493450164795
Reality:  REAL ID:  11  Label:  neutral  Accuracy:  0.8577689528465271
Reality:  FAKE ID:  12  Label:  neutral  Accuracy:  0.9211499094963074
Reality:  FAKE ID:  13  Label:  neutral  Accuracy:  0.7224054932594299
Reality:  REAL 

KeyboardInterrupt: 

Need to fine-tune the model using the fake-news data set too, not sure exactly how will look into it

Once decided on a model, need to make a new CSV with a column indicating its sentiment next to REAL or FAKE, I think making it a number rather than positive, negative, neutral is best idea.